# Analysis of Bioacoustic Data

This notebook provides tools for analyzing data using a custom classifier (developed with `agile_modeling.ipynb`).

In [0]:
#@title Installation. { vertical-output: true }
#@markdown You will likely need to work with `embed_audio.ipynb` and/or
#@markdown `agile_modeling.ipynb` before working with this notebook.
#@markdown
#@markdown Run this notebook in Google Colab by following
#@markdown [this link](https://colab.research.google.com/github/google-research/perch/blob/main/agile_modeling.ipynb).
#@markdown
#@markdown Run this cell to install the project dependencies.
%pip install git+https://github.com/google-research/perch.git


In [0]:
#@title Imports. { vertical-output: true }

import collections
from etils import epath
from ml_collections import config_dict
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from chirp.inference import colab_utils
colab_utils.initialize(use_tf_gpu=True, disable_warnings=True)

from chirp.inference import baw_utils
from chirp.inference import interface
from chirp.inference import tf_examples
from chirp.inference.search import bootstrap
from chirp.inference.search import search
from chirp.inference.search import display
from chirp.inference.classify import classify


In [0]:
#@title Basic Configuration. { vertical-output: true }

data_source = 'filesystem'  #@param['filesystem', 'a2o'] {type:'string'}
baw_auth_token = '' #@param

#@markdown Define the model: Usually perch or birdnet.
model_choice = 'perch'  #@param {type:'string'}
#@markdown Set the base directory for the project.
working_dir = '/tmp/agile'  #@param {type:'string'}

# Set the embedding and labeled data directories.
labeled_data_path = epath.Path(working_dir) / 'labeled'
custom_classifier_path = epath.Path(working_dir) / 'custom_classifier'

# The embeddings_path should be detected automatically, but can be overridden.
embeddings_path = ''


In [0]:
#@title Load Existing Project State and Models. { vertical-output: true }

if data_source == 'a2o':
  embedding_config = baw_utils.get_a2o_embeddings_config()
  bootstrap_config = bootstrap.BootstrapConfig.load_from_embedding_config(
      embedding_config=embedding_config,
      annotated_path=labeled_data_path,
      embeddings_glob='*/embeddings-*')
  embeddings_path = embedding_config.output_dir
elif (embeddings_path
      or (epath.Path(working_dir) / 'embeddings/config.json').exists()):
  if not embeddings_path:
    # Use the default embeddings path, as it seems we found a config there.
    embeddings_path = epath.Path(working_dir) / 'embeddings'
  # Get relevant info from the embedding configuration.
  bootstrap_config = bootstrap.BootstrapConfig.load_from_embedding_path(
      embeddings_path=embeddings_path,
      annotated_path=labeled_data_path)
else:
  raise ValueError('No embedding configuration found.')

project_state = bootstrap.BootstrapState(
    bootstrap_config, baw_auth_token=baw_auth_token)

cfg = config_dict.ConfigDict({
    'model_path': custom_classifier_path,
    'logits_key': 'custom',
})
logits_head = interface.LogitsOutputHead.from_config(cfg)
model = logits_head.logits_model
class_list = logits_head.class_list
print('Loaded custom model with classes: ')
print('\t' + '\n\t'.join(class_list.classes))

In [0]:
#@title Write classifier inference CSV. { vertical-output: true }

#@markdown This cell writes detections (locations of audio windows where
#@markdown the logit was greater than a threshold) to a CSV file.

output_filepath = epath.Path(working_dir) / 'inference.csv'  #@param

#@markdown Set the default detection thresholds, used for all classes.
#@markdown To set per-class detection thresholds, modify the code below.
#@markdown Keep in mind that thresholds are on the logit scale, so 0.0
#@markdown corresponds to a 50% model confidence.
default_threshold = 0.0  #@param
if default_threshold is None:
  # In this case, all logits are written. This can lead to very large CSV files.
  class_thresholds = None
else:
  class_thresholds = collections.defaultdict(lambda: default_threshold)
  # Set per-class thresholds here.
  class_thresholds['my_class'] = 1.0

#@markdown Classes to ignore when counting detections.
exclude_classes = ['unknown']  #@param

#@markdown The `include_classes` list is ignored if empty.
#@markdown If non-empty, only scores for these classes will be written.
include_classes = []  #@param

embeddings_ds = tf_examples.create_embeddings_dataset(
    embeddings_path, file_glob='embeddings-*')

classify.write_inference_csv(
    embeddings_ds=embeddings_ds,
    model=logits_head,
    labels=class_list.classes,
    output_filepath=output_filepath,
    threshold=class_thresholds,
    embedding_hop_size_s=bootstrap_config.embedding_hop_size_s,
    include_classes=include_classes,
    exclude_classes=exclude_classes)

## Call Density Estimation

See 'All Thresholds Barred': https://arxiv.org/abs/2402.15360

In [0]:
#@title Validation and Call Density. { vertical-output: true }

target_class = 'my_class'  #@param {type:'string'}

#@markdown Bin bounds for validation. Should be an ordered list, beginning with
#@markdown 0.0 and ending with 1.0.
bounds = [0.0, 0.9, 0.99, 0.999, 1.0]  #@param
#@markdown Number of validation samples per bin.
samples_per_bin = 25  #@param

bounds = np.array(bounds)
num_bins = len(bounds) - 1

# Select `top_k`` so that we are reasonably sure to get at least samples_per_bin
# samples in the rarest bin in a randomly selected set of `top_k` examples.
bin_probs = bounds[1:] - bounds[:-1]
rarest_prob = np.min(bin_probs)
top_k = int(samples_per_bin  / rarest_prob * 2)

embeddings_ds = project_state.create_embeddings_dataset(shuffle_files=True)
results, all_logits = search.classifer_search_embeddings_parallel(
    embeddings_classifier=logits_head,
    target_index=class_list.classes.index(target_class),
    random_sample=True,
    top_k=top_k,
    hop_size_s=bootstrap_config.embedding_hop_size_s,
    embeddings_dataset=embeddings_ds,
)

q_bounds = np.quantile(all_logits, bounds)
binned = [[] for _ in range(num_bins)]
for r in results.search_results:
  result_bin = np.argmax(r.score < q_bounds) - 1
  binned[result_bin].append(r)
binned = [np.random.choice(b, samples_per_bin, replace=False) for b in binned]

combined_results = []
for b in binned:
  combined_results.extend(b)
rng = np.random.default_rng(42)
rng.shuffle(combined_results)

ys, _, _, = plt.hist(all_logits, bins=100, density=True)
for q in q_bounds:
  plt.plot([q, q], [0.0, np.max(ys)], 'k:', alpha=0.75)
plt.show()


In [0]:
#@title Display Results. { vertical-output: true }

samples_per_page = 40  #@param
page_state = display.PageState(
    np.ceil(len(combined_results) / samples_per_page))

display.display_paged_results(
    search.TopKSearchResults(len(combined_results), combined_results),
    page_state, samples_per_page,
    project_state=project_state,
    embedding_sample_rate=project_state.embedding_model.sample_rate,
    exclusive_labels=True,
    checkbox_labels=[target_class, f'not {target_class}', 'unsure'],
)

In [0]:
#@title Collate results and write validation log. { vertical-output: true }

validation_log_filepath = (
    epath.Path(working_dir) / f'validation_{target_class}.csv')

filenames = []
timestamp_offsets = []
scores = []
is_pos = []
weights = []
bins = []

for r in combined_results:
  if not r.label_widgets: continue
  value = r.label_widgets[0].value
  if value is None:
    continue
  filenames.append(r.filename)
  scores.append(r.score)
  timestamp_offsets.append(r.timestamp_offset)

  # Get the bin number and sampling weight for the search result.
  result_bin = np.argmax(r.score < q_bounds) - 1
  bins.append(result_bin)
  weights.append(bin_probs[result_bin])

  if value == target_class:
    is_pos.append(1)
  elif value == f'not {target_class}':
    is_pos.append(-1)
  elif value == 'unsure':
    is_pos.append(0)

label = [target_class for _ in range(len(filenames))]
log = pd.DataFrame({
    'filenames': filenames,
    'timestamp_offsets': timestamp_offsets,
    'scores': scores,
    'is_pos': is_pos,
    'weights': weights,
    'bins': bins,
})
log.to_csv(validation_log_filepath, mode='a')

In [0]:
#@title Estimate Call Density. { vertical-output: true }

import scipy

# Collect validated labels by bin.
bin_pos = [0 for i in range(num_bins)]
bin_neg = [0 for i in range(num_bins)]
for score, pos in zip(scores, is_pos):
  result_bin = np.argmax(score < q_bounds) - 1
  if pos == 1:
    bin_pos[result_bin] += 1
  elif pos == -1:
    bin_neg[result_bin] += 1

# Create beta distributions.
prior = 0.1
betas = [scipy.stats.beta(p + prior, n + prior)
         for p, n in zip(bin_pos, bin_neg)]
# MLE positive rate in each bin.
mle_b = np.array([bin_pos[b] / (bin_pos[b] + bin_neg[b] + 1e-6)
                  for b in range(num_bins)])

# Probability of each bin, P(b).
p_b = bin_probs

# MLE total call density.
q_mle = np.dot(mle_b, p_b)

num_beta_samples = 10_000
q_betas = []
for _ in range(num_beta_samples):
  qs_pos = np.array([b.rvs(size=1)[0] for b in betas])  # P(+|b)
  q_beta = np.dot(qs_pos, p_b)
  q_betas.append(q_beta)

# Plot call density estimate.
plt.figure(figsize=(10, 5))
xs, ys, _ = plt.hist(q_betas, density=True, bins=25, alpha=0.25)
plt.plot([q_mle, q_mle], [0.0, np.max(xs)], 'k:', alpha=0.75,
         label='q_mle')

low, high = np.quantile(q_betas, [0.05, 0.95])
plt.plot([low, low], [0.0, np.max(xs)], 'g', alpha=0.75, label='low conf')
plt.plot([high, high], [0.0, np.max(xs)], 'g', alpha=0.75, label='high conf')

plt.xlim(0.0, 1.0)
plt.xlabel('Call Rate (q)')
plt.ylabel('P(q)')
plt.title(f'Call Density Estimation ({target_class})')
plt.legend()
plt.show()

print(f'MLE Call Density: {q_mle:.4f}')
print(f'(Low/MLE/High) Call Density Estimate: ({low:5.4f} / {q_mle:5.4f} / {high:5.4f})')


In [0]:
#@title Naive Estimation of ROC-AUC for target class. { vertical-output: true }
#@markdown Computes ROC-AUC from the validation logs, with bin weighting.
#@markdown ROC-AUC is the overall probability that a random positive example
#@markdown has a higher classifier score than a random negative example.

# Probability of bins
p_b = bin_probs

bin_pos, bin_neg = np.array(bin_pos), np.array(bin_neg)
# Compute P(z(+) > z(-))
# P(b_i|+) * P(b_k|-)
n_pos = np.sum(bin_pos)
n_neg = np.sum(bin_neg)
p_pos_b = np.array(bin_pos) / (bin_pos + bin_neg)
p_neg_b = np.array(bin_neg) / (bin_pos + bin_neg)
p_pos = np.sum(p_pos_b * p_b)
p_neg = np.sum(p_neg_b * p_b)

p_b_pos = p_pos_b * p_b / p_pos
p_b_neg = p_neg_b * p_b / p_neg
roc_auc = 0
# For off-diagonal bin pairs:
# Take the probability of drawing a pos from bin j and neg from bin i.
# If j > i, all pos examples are scored higher, so contributes directly to the
# total ROC-AUC.
for i in range(num_bins):
  for j in range(i + 1, num_bins):
    roc_auc += p_b_pos[j] * p_b_neg[i]

# For diagonal bin-pairs:
# Look at actual in-bin observations for diagonal contribution.
bins = np.array(bins)
is_pos = np.array(is_pos)

for b in range(num_bins):
  bin_pos_idxes = np.argwhere((bins == b) * (is_pos == 1))[:, 0]
  bin_neg_idxes = np.argwhere((bins == b) * (is_pos == -1))[:, 0]
  bin_pos_scores = np.array(scores)[bin_pos_idxes]
  bin_neg_scores = np.array(scores)[bin_neg_idxes]
  if bin_pos_scores.size == 0:
    continue
  if bin_neg_scores.size == 0:
    continue
  # Count total number of pairs where the pos examples have a higher score than
  # a negative example.
  hits = ((bin_pos_scores[:, np.newaxis]
           - bin_neg_scores[np.newaxis, :]) > 0).sum()
  bin_roc_auc = hits / (bin_pos_scores.size * bin_neg_scores.size)
  # Contribution is the probability of pulling both pos and neg examples
  # from this bin, multiplied by the bin's ROC-AUC.
  roc_auc += bin_roc_auc * p_b_pos[b] * p_b_neg[b]

print(f'ROC-AUC : {roc_auc:.3f}')